In [1]:
import time
import numpy as np
import tensorflow as tf
import sys
import os

In [2]:
os.chdir('G:\\server\\models\\tutorials\\rnn\\ptb')
import reader

In [18]:
class PTBInput(object):
    def __init__(self ,config, data, name = None):
        self.batch_size = batch_size = config.batch_size
        self.num_steps = num_steps = config.num_steps
        self.epoch_size = ((len(data)//batch_size) - 1)// num_steps
        self.input_data, self.targets = reader.ptb_producer(
            data, batch_size , num_steps, name = name
        )
class PTBModel(object):
    def __init__(self, is_training, config, input_):
        self._input = input_
        batch_size = input_.batch_size
        num_steps = input_.num_steps
        size = config.hidden_size
        vocab_size = config.vocab_size
        def lstm_cell():
            return tf.contrib.rnn.BasicLSTMCell(
                size, forget_bias = 0.0, state_is_tuple = True
            )
        attn_cell = lstm_cell
        if is_training and config.keep_prob < 1:
            def attn_cell():
                return tf.contrib.rnn.DropoutWrapper(
                    lstm_cell(), output_keep_prob = config.keep_prob
                )
        cell = tf.contrib.rnn.MultiRNNCell(
            [attn_cell() for _ in range(config.num_layers)],
            state_is_tuple = True
        )
        self._initial_state = cell.zero_state(batch_size, tf.float32)
        with tf.device('/cpu:0'):
            embedding = tf.get_variable(
                'embedding', [vocab_size, size], dtype = tf.float32
            )
            inputs = tf.nn.embedding_lookup(embedding, input_.input_data)
        if is_training and config.keep_prob < 1:
            inputs = tf.nn.dropout(inputs, config.keep_prob)
        outputs = []
        state = self._initial_state
        with tf.variable_scope('RNN'):
            for time_step in range(num_steps):
                if time_step > 0 :tf.get_variable_scope().reuse_variables()
                (cell_output, state) = cell(inputs[:, time_step, :], state)
                outputs.append(cell_output)
        output = tf.reshape(tf.concat(outputs, 1), [-1, size])
        softmax_w = tf.get_variable('softmax_w', [size, vocab_size], dtype= tf.float32)
        softmax_b = tf.get_variable('softmax_b', [vocab_size], dtype= tf.float32)
        logits = tf.matmul(output, softmax_w) + softmax_b
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            [logits], [ tf.reshape(input_.targets, [-1])], [tf.ones([batch_size * num_steps], dtype= tf.float32
        )])
        self._cost = cost = tf.reduce_sum(loss) / batch_size
        self._fianl_state = state
        if not is_training:
            return
        self._lr = tf.Variable(0.0, trainable= False)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),config.max_grad_norm)
        optimizer = tf.train.GradientDescentOptimizer(self._lr)
        self._train_op = optimizer.apply_gradients(zip(grads, tvars), global_step= tf.contrib.framework.get_or_create_global_step())
        self._new_lr = tf.placeholder(tf.float32, shape = [], name = 'new_learning_rate')
        self._lr_update = tf.assign(self._lr, self._new_lr)
    def assign_lr(self, session, lr_value):
        session.run(self._lr_update, feed_dict = {self._new_lr: lr_value})
    @property
    def input(self):
        return self._input
    @property
    def initial_state(self):
        return self._initial_state
    @property
    def cost(self):
        return self._cost
    @property
    def final_state(self):
        return self._fianl_state
    @property
    def lr(self):
        return self._lr
    @property
    def train_op(self):
        return self._train_op

class SmallConfig(object):
    init_scale = 0.1
    learning_rate = 1.0
    max_grad_norm = 5
    num_layers = 2
    num_steps = 20
    hidden_size = 200
    max_epoch = 4
    max_max_epoch = 13
    keep_prob = 1.0
    lr_decay = 0.5
    batch_size = 20
    vocab_size = 10000
class MediunConfig(object):
    init_scale = 0.05
    learning_rate = 1.0
    max_grad_norm = 5
    num_layers = 2
    num_steps = 35
    hidden_size = 650
    max_epoch = 6
    max_max_epoch = 39
    keep_prob = 0.5
    lr_decay = 0.8
    batch_size = 20
    vocab_size = 10000
class LargeConfig(object):
    init_scale = 0.04
    learning_rate = 1.0
    max_grad_norm = 10
    num_layers = 2
    num_steps = 35
    hidden_size = 1500
    max_epoch = 14
    max_max_epoch = 55
    keep_prob = 0.35
    lr_decay = 1 / 1.15
    batch_size = 20
    vocab_size = 10000
class TestConfig(object):
    init_scale = 0.1
    learning_rate = 1.0
    max_grad_norm = 1
    num_layers = 1
    num_steps = 2
    hidden_size = 2
    max_epoch =1
    max_max_epoch = 1
    keep_prob = 1.0
    lr_decay = 0.5
    batch_size = 20
    vocab_size = 10000

def run_epoch(session, model, eval_op = None, verbose = False):
    start_time = time.time()
    costs = 0.0
    iters = 0
    state = session.run(model.initial_state)
    fetchs = {
        'cost' : model.cost,
        'final_state': model.final_state
    }
    if eval_op is not None:
        fetchs['eval_op'] = eval_op
    for step in range(model.input.epoch_size):
        feed_dict = {}
        for i, (c,h) in enumerate(model.initial_state):
            feed_dict[c] = state[i].c
            feed_dict[h] = state[i].h
        vals = session.run(fetchs, feed_dict)
        cost = vals['cost']
        state = vals['final_state']
        costs += cost
        iters += model.input.num_steps

        if verbose and step%(model.input.epoch_size // 10) ==10:
            print('%.3f perlexity : %.3f speed : %.0f wps'
                  % (step * 1.0/model.input.epoch_size, np.exp(costs/ iters),
                     iters * model.input.batch_size/(time.time() - start_time)))
    return np.exp(costs / iters)

In [4]:
os.getcwd()

'G:\\server\\models\\tutorials\\rnn\\ptb'

In [5]:
raw_data = reader.ptb_raw_data('simple-examples/data/')
train_data, valid_data, test_data, _= raw_data

config = SmallConfig()
eval_config = SmallConfig()
eval_config.batch_size = 1
eval_config.num_steps = 1

In [20]:
with tf.Graph().as_default():
    initializer = tf.random_uniform_initializer(-config.init_scale, config.init_scale)
    with tf.name_scope('Train'):
        
        train_input = PTBInput(config= config, data= train_data, name = 'TrainInput')
        with tf.variable_scope('Model', reuse= None, initializer = initializer):
            m = PTBModel(is_training= True, config= config, input_= train_input)
    with tf.name_scope('valid'):
        valid_input = PTBInput(config = config, data = valid_data, name= 'ValidInput')
        with tf.variable_scope('Model', reuse= True, initializer= initializer):
            mvalid = PTBModel(is_training = False, config= config, input_= valid_input)
    with tf.name_scope('Test'):
        test_input  = PTBInput(config= eval_config, data = test_data, name = 'TestInput')

        with tf.variable_scope('Model', reuse= True, initializer= initializer):
            mtest = PTBModel(is_training = False, config= eval_config, input_ = test_input)
            
    sv = tf.train.Supervisor()
    with sv.managed_session() as session:
        for i in range(config.max_max_epoch):
            lr_decay = config.lr_decay ** max(i + 1 - config.max_epoch, 0.0)
            m.assign_lr(session, config.learning_rate * lr_decay)
            print('Epoch: %d Learning rate : %.3f' %(i+1, session.run(m.lr)))
            train_perlexity = run_epoch(session, m, eval_op= m.train_op, verbose= True)
            print('Epoch: %d Train Perplexity: %.3f' % (i+1, train_perlexity))
            valid_perlexity = run_epoch(session, mvalid)
            print('Epoch: %d Valid Perplexity: %.3f' % (i+1, valid_perlexity))
            test_perlexity = run_epoch(session, mtest)
            print('Test Perplexity: %.3f' % (test_perlexity))

INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.
Epoch: 1 Learning rate : 1.000
0.004 perlexity : 5523.933 speed : 604 wps
0.104 perlexity : 829.211 speed : 626 wps
0.204 perlexity : 615.099 speed : 586 wps
0.304 perlexity : 497.713 speed : 649 wps
0.404 perlexity : 429.386 speed : 682 wps
0.504 perlexity : 384.820 speed : 710 wps
0.604 perlexity : 346.747 speed : 727 wps
0.703 perlexity : 320.620 speed : 743 wps
0.803 perlexity : 299.979 speed : 752 wps
0.903 perlexity : 281.047 speed : 760 wps
Epoch: 1 Train Perplexity: 266.820
Epoch: 1 Valid Perplexity: 179.431
Test Perplexity: 176.435
Epoch: 2 Learning rate : 1.000
0.004 perlexity : 211.132 speed : 860 wps
0.104 perlexity : 151.064 speed : 840 wps
0.204 perlexity : 158.024 speed : 835 wps
0.304 perlexity : 152.821 speed : 835 wps
0.404 perlexity : 149.998 speed : 842 wps
0.504 perlexity : 147.410 speed : 843 wps
0.604 perlexity : 142.761 speed : 843 wps
0.703 perlexity : 140.676 speed : 840 wps
0.